In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# O que é o unsloth? O Unsloth torna o ajuste fino de grandes modelos de linguagem como Llama-3, Mistral, Phi-3 e Gemma 2x mais rápido, usa 70% menos memória e sem degradação na precisão!
# peft accelerate bitsandbytes: melhoram o processo de treinamento de grandes modelos onde há restrições de hardwares.

!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-yh3ac2z_/unsloth_6341ec61e4904af4b2231bab13ef520a
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-yh3ac2z_/unsloth_6341ec61e4904af4b2231bab13ef520a
  Resolved https://github.com/unslothai/unsloth.git to commit 85f1fa096afde5efe2fb8521d8ceec8d13a00715
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 87.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.1/112.1 kB 9.6 MB/s eta 0:00:00
   ━━━

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 85.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 19.5 MB/s eta 0:00:00
  Attempting uninstall: trl
    Found existing installation: trl 0.13.0
    Uninstalling trl-0.13.0:
      Successfully uninstalled trl-0.13.0


In [6]:
# Importações necessárias para treinamento, carregamento do dataset

from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
import json
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from transformers import TextStreamer

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [22]:
# Declaração de variáveis necessárias para o código, como caminho do dataset.
# A variável load_in_4bit = True indica que será usada a técnica de quantização para reduzir ainda mais o uso de memória.

DATA_PATH = "/content/drive/MyDrive/FIAP_TECH_CHALLENGE_III/QWEN2_5/Qwen2.5/json_files/trn.json"
OUTPUT_PATH_DATASET = "/content/drive/MyDrive/FIAP_TECH_CHALLENGE_III/QWEN2_5/Qwen2.5/json_files/new_formatted_news_dataset_trn.json"
max_seq_length = 2048
dtype = None
load_in_4bit = True

In [15]:
# Etapa de preparação do dataset: método que criar um novo dataset a partir do dataset original removendo linhas com descrição vazia. Utiliza apenas as colunas Title e content

def format_dataset_into_model_input(input_file):
    processed_data = []

    # Abrir o arquivo original
    with open(input_file, "r") as file:
        for line in file:
            # Ler cada linha como um JSON
            record = json.loads(line)

            # Ignorar entradas onde o campo 'content' está vazio
            if not record["content"].strip():
                continue

            # Criar os prompts no formato esperado
            processed_data.append({
                "instruction": "Generate a product description based on the provided title as input",
                "input": f"Title: {record['title']}",
                "output": f"Description: {record['content']}"  # Empty response for fine-tuning
            })

    # Salvando o resultado em um arquivo JSON
    with open(OUTPUT_PATH_DATASET, 'w') as output_file:
        json.dump(processed_data, output_file, indent=4,ensure_ascii=False)

    print(f"Dataset salvo em {OUTPUT_PATH_DATASET}")

In [16]:
# Execução do método acima para a a criação do novo arquivo do dataset (json)

format_dataset_into_model_input(input_file=DATA_PATH)

Dataset salvo em /content/drive/MyDrive/FIAP_TECH_CHALLENGE_III/QWEN2_5/Qwen2.5/json_files/new_formatted_news_dataset_trn.json


In [4]:
# Carrega um modelo pré-treinado do unsloth já com otimização para maior velocidade e eficiência. Não utiliza diretamente do hugging face
# Neste momento definimos o uso do modelo Qwen2.5-7B.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-7B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

NameError: name 'FastLanguageModel' is not defined

In [18]:
# PEFT (Parameter-Efficient Fine-Tuning): Técnica que atualiza apenas um sub-conjunto de parâmetros e não o modelo inteiro, tornando mais rápido e eficiente.
# Economia de tempo, memória e recurso computacional
# Este trecho foca mais no desempenho em locais onde GPU são limitadas.

model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth",
    random_state = 42,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.12.4 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [19]:
# Criação de um pipeline para formatar os prompts do dataset. O foco do alpaca é instruções.

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}
"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

dataset = load_dataset("json", data_files=OUTPUT_PATH_DATASET, split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1498718 [00:00<?, ? examples/s]

In [ ]:
# Treinamento do modelo com alguns argumentos importantes para desempenho e otimização do processo de fine-tuning.
# per_device_train_batch_size e gradient_accumulation_steps otimizam o uso de memória no ambiente com GPU
# optim = "adamw_8bit" e weight_decay são ideais para otimizar treinamento de modelos grandes

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 4,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 8,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        dataloader_num_workers=4,
    ),
)

Map (num_proc=4):   0%|          | 0/1498718 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,498,718 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 8 | Gradient Accumulation steps = 4
\        /    Total batch size = 32 | Total steps = 60
 "-____-"     Number of trainable parameters = 40,370,176


Step,Training Loss
1,2.502800
2,2.364500
3,2.449000
4,2.338500
5,2.431800
6,2.335400
7,1.992500
8,2.129200
9,1.986100
10,2.102000


In [9]:
# Nesta etapa estamos preparando o modelo para inferência
# Utiliza os tensores cuda para acelerar o processamento
# Criação do prompt usando alpaca

FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Generate a product description based on the provided title as input", # instruction
        "Title: It's About Time, Jesse Bear: And Other Rhymes", # input
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 512, use_cache = True)
tokenizer.batch_decode(outputs)

["Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nGenerate a product description based on the provided title as input\n\n### Input:\nTitle: It's About Time, Jesse Bear: And Other Rhymes\n\n### Response:\n\nIt's About Time, Jesse Bear: And Other Rhymes is a delightful collection of children's rhymes that will captivate young readers with its charming illustrations and engaging stories. This book is perfect for parents and educators who want to introduce their children to the world of poetry and storytelling. With its fun and playful language, it's sure to become a favorite among kids of all ages. So, if you're looking for a fun and educational way to introduce your child to the world of literature, look no further than It's About Time, Jesse Bear: And Other Rhymes.<|endoftext|>"]

In [ ]:
# Local onde o modelo será gravado após fine-tuning

model.save_pretrained("/content/drive/MyDrive/FIAP_TECH_CHALLENGE_III/QWEN2_5/Qwen2.5/qwen2.5_model") # Local saving
tokenizer.save_pretrained("/content/drive/MyDrive/FIAP_TECH_CHALLENGE_III/QWEN2_5/Qwen2.5/qwen2.5_model")

('./outputs/qwen2.5_model/tokenizer_config.json',
 './outputs/qwen2.5_model/special_tokens_map.json',
 './outputs/qwen2.5_model/vocab.json',
 './outputs/qwen2.5_model/merges.txt',
 './outputs/qwen2.5_model/added_tokens.json',
 './outputs/qwen2.5_model/tokenizer.json')

In [25]:
# Neste momento iremos usar o modelo já com fine-tuning

model, tokenizer = FastLanguageModel.from_pretrained(
	model_name = "/content/drive/MyDrive/FIAP_TECH_CHALLENGE_III/QWEN2_5/Qwen2.5/qwen2.5_model",
	max_seq_length = max_seq_length,
	dtype = dtype,
	load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2024.12.4: Fast Qwen2 patching. Transformers:4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(152064, 3584, padding_idx=151665)
        (layers): ModuleList(
          (0-27): 28 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3584, out_features=3584, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3584, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=3584, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora

In [2]:
# Testando o modelo com FINE-TUNING

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}
"""

inputs = tokenizer(
[
    alpaca_prompt.format(
        "Generate a product description based on the provided title as input", # instruction
        "Title: If You Were There in 1776", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs,
                   streamer = text_streamer,
                   max_new_tokens = 1024,
                   temperature=0.3,  # Menor aleatoriedade
                   top_p=0.9,        # Probabilidade acumulada
                   top_k=50 )

NameError: name 'tokenizer' is not defined